In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
#allows to import generator and discriminator
!pip install -q git+https://github.com/tensorflow/examples.git

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
from os import listdir
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import vstack
from numpy import asarray
from numpy import savez_compressed
import numpy as np
from PIL import Image
from tensorflow.keras.utils import plot_model

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn import preprocessing

#AUTOTUNE = tf.data.AUTOTUNE
AUTOTUNE = tf.data.experimental.AUTOTUNE

from PIL import Image
import glob
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
tf.version.VERSION

# <font color='red'>**Useful methods**</font>

In [ ]:
# load all images in a directory into memory
def load_images(path, size=(256,256)):
    data_list = list()
    #enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        # convert to numpy array
        pixels = img_to_array(pixels)
        # store
        data_list.append(pixels)
    return asarray(data_list)

In [ ]:
def generate_images(model, test_input, c, month, folder, label):
    
    pair_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/pair_images/'
    if not os.path.exists(pair_path):
        os.makedirs(pair_path)
    
    wl_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/realWL/'
    if not os.path.exists(wl_path):
        os.makedirs(wl_path)
    
    artif_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/artifNBI/'
    if not os.path.exists(artif_path):
        os.makedirs(artif_path)
    
    
    prediction = model(test_input)
    display_list = [test_input[0], prediction[0]]
    
    plt.figure(figsize=(12, 12))
    
    #saving singles images
    
    #real WL
    name_file = wl_path + label + '_img_' + str(c) + '.png'
    # getting the pixel values between [0, 1] to plot it.
    tf.keras.preprocessing.image.save_img(name_file, display_list[0] * 0.5 + 0.5)
    
    
    #artif NBI
    name_file = artif_path + label + '_img_' + str(c) + '.png'
    # getting the pixel values between [0, 1] to plot it.
    tf.keras.preprocessing.image.save_img(name_file, display_list[1] * 0.5 + 0.5)

    title = ['Input Image', 'Predicted Image']
        
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
        
    name_file = pair_path + label + '_img_' + str(c) + '.png'
    plt.savefig(name_file, format='png')
    plt.show()

**Data augmentation techniques**

In [ ]:
def random_crop(image):
    cropped_image = tf.image.random_crop(image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image

# scaling the images to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

**Preprocess splits**

In [ ]:
def preprocess_image_train(image):
    image = random_jitter(image)
    image = normalize(image)
    return image

def preprocess_image_test(image):
    image = normalize(image)
    return image

**Import and reuse the Pix2Pix models**

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

**Initializing optimizers, generator and discriminators**

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

nbi_cls_model_optimizier = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# <font color='red'>**Loading models**</font>

In [ ]:
base_model = tf.keras.models.load_model('../models/classifier/binary/MobileNet.h5', compile=True)
print("model loaded!")

In [ ]:
for layer in base_model.layers:
    print(layer.name)

In [ ]:
backbone = base_model.get_layer('mobilenet_1.00_224')
x = base_model.get_layer('global_average_pooling2d')(backbone.output)
x = base_model.get_layer('dense')(x)
x = base_model.get_layer('dropout')(x)
x = base_model.get_layer('dense_1')(x)

nbi_cls_model = tf.keras.Model(inputs=backbone.input, outputs=x)
print(nbi_cls_model.summary())

In [ ]:
checkpoint_path = "../models/lossweighted/"
ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer,
                           nbi_cls_model=nbi_cls_model)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')
ckpt.restore(ckpt_manager.latest_checkpoint)
if ckpt_manager.latest_checkpoint:
    print("Restored from {}".format(ckpt_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

In [ ]:
generator_g.summary()

In [ ]:
plot_model(generator_g, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## **testing single video**

In [ ]:
def generate_images(model, test_input):
    prediction = model(test_input)
    plt.figure(figsize=(12, 12))
    
    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
# frames from video path
path =  "../../../../../data/polyp_original/WL/adenoma_WL/video_1/"
# load dataset white light
# here A: white light, B: nbi light
adenoma_WL = load_images(path)
print("Adenoma WL video_1: ", adenoma_WL.shape)

In [ ]:
adenoma_WL_array = np.asarray(adenoma_WL)
adenoma_WL_ds = tf.data.Dataset.from_tensor_slices(adenoma_WL_array)
adenoma_WL_ds = adenoma_WL_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
                BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# Run the trained model on the test dataset
for inp in adenoma_WL_ds.take(adenoma_WL.shape[0]):
    generate_images(generator_g, inp)

# <font color='red'>**IGHO predicting**</font>

Nota: septiembre esta en proceso

In [ ]:
month = "SEPTIEMBRE"
txt_folder = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + 'folders.txt'
with open(txt_folder) as f:
    content = f.readlines()
    
folders = [cont.split('\n')[0] for cont in content]
print(folders)

In [ ]:

folder = folders[0]
label = 'adenoma'
#test
txt_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/' + 'polyps.txt'

pathImgs = "/../Datasets/Igho/Uncompressed/Videos/"+ month + "/" + folder + "/Normal/"

with open(txt_path) as f:
    content = f.readlines()

frames = [cont.split('.')[0] for cont in content]

data_list = list()
size=(256,256)
for frame in frames:
    to_read = pathImgs + frame + '.png'
    pixels = load_img(to_read, target_size=size)
    # convert to numpy array
    pixels = img_to_array(pixels)
    # store
    data_list.append(pixels)

WL = asarray(data_list)

print("The folder: {} has a shape of: {}".format(folder,WL.shape))

In [ ]:
WL_array = np.asarray(WL)
WL_ds = tf.data.Dataset.from_tensor_slices(WL_array)
WL_ds = WL_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
                BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
for index, inp in zip(frames, WL_ds.take(WL.shape[0])):
    generate_images(generator_g, inp, index, month, folder, label)

# <font color='red'>**Predicting over full frames videos**</font>

In [ ]:
def generatingImages(data, generator, salve_path):
    
    for count, inp in enumerate(data):
        fake = generator(inp)
        fake = fake[0]* 0.5 + 0.5
        #para que PIL Image deje guardar (mult por 255 and change by uint8)
        fake = np.array(fake) * 255
        fake = fake.astype(np.uint8)
        fake_img = Image.fromarray(fake)
        
        #checking if directory exist
        directory = salve_path.split('/')[:-1]
        directory = "/".join(directory)
        if not os.path.exists(directory):
            os.mkdir(directory)
        
        clase = directory.split('/')[-2:]
        clase = "_".join(clase)
        salve_path = directory + '/' + clase + '_img_' + str(count) + '.png'
        print("to save: ", salve_path)
        fake_img.save(salve_path)    

In [ ]:
def toDataSet(path_origen):
    data = load_images(path_origen)
    print("dimensiones de data: ", data.shape)
    data_array = np.asarray(data)
    data_ds = tf.data.Dataset.from_tensor_slices(data_array)
    data_ds = data_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
                    BUFFER_SIZE).batch(BATCH_SIZE)
    
    return data_ds

In [ ]:
# frames from video path
path = '/../../../Tesis/Polipos/DataSet/PolypDatasetProcesed/WL/'
save = '/../../../maestria/propuesta/experiments/data/wl2nbi/fake_images_ck80/'
clases = os.listdir(path)
for clase in clases:
    print("========= clase ", clase, "=========")
    clase_pth = path + clase + '/'
    videos = os.listdir(clase_pth)
    for video in videos:
        print("===",video, "====")
        video_pth = clase_pth + video + '/'
        print("convirtiendo a tf.Dataset...")
        data_ds = toDataSet(video_pth)
        imgs = os.listdir(video_pth)
        print("prediciendo con cycleGan...")
        path_save = save + clase + '/' + video + '/'
        generatingImages(data_ds, generator_g, path_save)
            

print("Finalizado!")

# <font color='red'>**Predicting over full test frame videos**</font>

In [ ]:
path = '../../../data/binary/test_WL/'
videos = set()
for file in os.listdir(path):
    clase = file.split('_')[0]
    video = file.split('_')[-3]
    name = clase + '_WL/video_' + video
    videos.add(name)

videos = list(videos)
print(videos)

In [ ]:
#for serrated samples
path = '../../../../../data/polyp_original/WL/serrated_WL/'
videos = set()
for file in os.listdir(path):
    name = 'serrated_WL/' + file
    videos.add(name)

videos = list(videos)
print(videos)

In [ ]:
def generatingImages(data, generator, salve_path):
    
    for count, inp in enumerate(data):
        fake = generator(inp)
        fake = fake[0]* 0.5 + 0.5
        #para que PIL Image deje guardar (mult por 255 and change by uint8)
        fake = np.array(fake) * 255
        fake = fake.astype(np.uint8)
        fake_img = Image.fromarray(fake)
        
        #checking if directory exist
        directory = salve_path.split('/')[:-1]
        directory = "/".join(directory)
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        clase = directory.split('/')[-2:]
        clase = "_".join(clase)
        salve_path = directory + '/' + clase + '_img_' + str(count) + '.png'
        fake_img.save(salve_path)    

In [ ]:
def toDataSet(path_origen):
    
    #Make a tf dataSet for images and labels    
    data = load_images(path_origen)
    data_array = np.asarray(data)
    data_ds = tf.data.Dataset.from_tensor_slices(data_array)
    data_ds = data_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE)
    
    
    return data_ds

In [ ]:
gen_path = '../../../../../data/polyp_original/WL/'
save = '../imgs_results/binary/lossweighted3_1_test/'

for video in videos:
    video_pth = gen_path + video + '/'
    print("convirtiendo a tf.Dataset...")
    data_ds = toDataSet(video_pth)
    #imgs = os.listdir(video_pth)
    print("prediciendo con cycleGan...")
    path_save = save + video + '/'
    generatingImages(data_ds, generator_g, path_save)

# <font color='red'>**Predicting over external data**</font>

In [ ]:
gen_path = '../../../../../../train/images/'
save = '../imgs_results/binary/lossweighted2/linaData/train/'

data_ds = toDataSet(gen_path)
print("prediciendo con cycleGan...")


for count, inp in enumerate(data_ds):
    fake = generator_g(inp)
    fake = fake[0]* 0.5 + 0.5
    #para que PIL Image deje guardar (mult por 255 and change by uint8)
    fake = np.array(fake) * 255
    fake = fake.astype(np.uint8)
    fake_img = Image.fromarray(fake)

    
    salve_path = save + 'img_' + str(count) + '.png'
    fake_img.save(salve_path)   
print("finished!")

# Creating video from fake frames predicted

In [ ]:
import cv2
import numpy as np
import glob

path = '/../../../../../../data/wl2nbi/fake_images_ck80/serrated_WL/video_2/*.png'

img_array = []
for filename in glob.glob(path):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('serV2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 7, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

# <font color='red'>**Predicting with netclassifier**</font>

**Making csv files for white light and synthetic nbi images**

In [ ]:
gen_path = '../../cycleGan-polyps/../../../../data/polyp_original/WL/'
gen_path2 = '../imgs_results/binary/lossweighted/'

paths = [gen_path, gen_path2]

for i, path in enumerate(paths):
    if i==0:
        csvfile = open('adeVsHypWhiteLight.csv', '+w')
    else:
        csvfile = open('adeVsHypArtifLossW2.csv', '+w')
    for video in videos:
        clase = video.split('/')[0]
        current_class = clase.split('_')[0]
        col_name = ','+ current_class + '\n'
        video_path = path + video
        images = os.listdir(video_path)
        for image in images:
            img_path = video_path + '/' + image
            csvfile.write(img_path+col_name)
csvfile.close()

In [ ]:
path = '../../../data/binary/test_NBI/'
files = os.listdir(path)
csvfile = open('adeVsHypNBI.csv', '+w')

for file in files:
    current_class = file.split('_')[0]
    col_name = ',' + current_class + '\n'
    img_path = path + file
    csvfile.write(img_path+col_name)
csvfile.close()

## Loading frames

In [ ]:
#test_df = pd.read_csv('adeVsHypNBI.csv', header=None)
test_df = pd.read_csv('../imgs_results/binary/lossweighted/adeVsHypArtifLossW1.csv', header=None)
test_df.columns = ['path', 'label']
test_df.head()

In [ ]:
test_df.groupby(['label']).count()

In [ ]:
test_df['path'].str.contains('video_11').sum()

to check: 
hyp_video1: 1988 real: 247
hyp_video3: 1173 real: 103
ade_video21: 1426 real: 1000

### Making generator

In [ ]:
def make_generator(df_test, HEIGHT, WIDTH, tipo, batch_size):
    
    test_datagen=ImageDataGenerator(rescale=1/255.0)
    
    test_generator=test_datagen.flow_from_dataframe(directory=None,
                                                    dataframe=df_test,
                                                    x_col='path',
                                                    y_col='label',
                                                    batch_size=batch_size,
                                                    seed=42,
                                                    shuffle=False,
                                                    class_mode=tipo,
                                                    target_size=(HEIGHT,WIDTH))

    return test_generator

In [ ]:
HEIGHT, WIDTH = 256, 256
tipo = 'categorical'
batch_size = 16

In [ ]:
test_gen = make_generator(test_df, HEIGHT, WIDTH, tipo, batch_size)

In [ ]:
#Confution Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, roc_auc_score
task = 'binary'

test_gen.reset()
logits = nbi_cls_model.predict_generator(test_gen, test_df.shape[0] // batch_size+1)
y_pred_class = np.argmax(logits, axis=1)
#predicted_class_probab=np.max(logits,axis=1)

if task=='binary':
    target_names = ['Adenoma', 'Hyperplastic']   
else:
    target_names = ['Adenoma', 'Hyperplastic', 'Serrated']    

print('Confusion Matrix')
print(confusion_matrix(test_gen.classes, y_pred_class))
print('Classification Report')
print(classification_report(test_gen.classes, y_pred_class, target_names=target_names))

In [ ]:
if task == 'binary':
    AUC = tf.keras.metrics.AUC()
    AUC.update_state(test_gen.classes, y_pred_class)
    gen_auc = AUC.result()
    print("AUC: ", gen_auc)
else:
    le = preprocessing.LabelEncoder()
    y_train_enc = le.fit_transform(test_gen.classes)
    y_test_enc = le.fit_transform(test_gen.classes)
    gen_auc = roc_auc_score(y_test_enc, logits, multi_class='ovr')
    print("AUC: ", gen_auc)